In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.eprbulletsafsc.com/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
from collections import defaultdict


anchor_tags = soup.find_all('a')
required_urls = defaultdict(list)
for tag in anchor_tags:
    href = tag['href']
    category = tag.text.split(' ')[0]

    cat_prefix = category[0] + 'XXXX'

    linked_page_url = url + "/" + href

    if not category[0].isdigit(): 
        required_urls['General'].append(['General', linked_page_url])
    else:
        required_urls[cat_prefix].append([category, linked_page_url])

# print(required_urls)
# print(required_urls['General'])

In [ ]:
res = []
required_count = 10
for category in required_urls:
    for sub_category, url in required_urls[category]:
        print(f"Processing {sub_category}")
        linked_page = requests.get(url)
        linked_soup = BeautifulSoup(linked_page.content, 'html.parser')
        divSoup = linked_soup.find("div", {"class" : "main"})
        ps = divSoup.findAll('p')

        count = 0
        for p in ps:
            splits = p.text.split('\n')

            for line in splits:
                if line != '': # Not having a check somehow leads to duplicates
                    res.append([category, sub_category, line[2:]])
                    count += 1
                if count == required_count: break
            if count == required_count: break

In [ ]:
import pandas as pd

# for key in res:
    # print(len(res[key]))

df = pd.DataFrame(res, columns=['Category', 'Sub Category', 'Data']).explode('Data').reset_index(drop=True)
print(df)
df.to_excel('output.xlsx', index=False)